In [ ]:
import os
import shutil

test1 = r"D:\Downloads\projectù\Tunisian license plate.v2i.yolov8\test\images"
train1 = r"D:\Downloads\projectù\Tunisian license plate.v2i.yolov8\train\images"
valid1 = r"D:\Downloads\projectù\Tunisian license plate.v2i.yolov8\valid\images"
test2 = r"D:\Downloads\projectù\tunisian-licensed-plates-DatasetNinja\test\img"
train2 = r"D:\Downloads\projectù\tunisian-licensed-plates-DatasetNinja\train\img"

sources = [test1, train1, valid1, test2, train2]
destination = r"D:\Downloads\projectù\all_images"
os.makedirs(destination, exist_ok=True)

for folder in sources:
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            src_path = os.path.join(folder, file)
            dst_path = os.path.join(destination, file)

            # Éviter l'écrasement des fichiers avec le même nom
            if os.path.exists(dst_path):
                name, ext = os.path.splitext(file)
                dst_path = os.path.join(destination, name + "_copy" + ext)

            shutil.copy(src_path, dst_path)



✅ Toutes les images ont été copiées avec succès.


In [3]:
import os

dossier = r"D:\Downloads\projectù\all_images"

nb_images = len([
    f for f in os.listdir(dossier)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

print("Nombre total d’images :", nb_images)


Nombre total d’images : 1105


In [10]:
import os
import json
import shutil

# -----------------------------
# Chemins du projet
# -----------------------------
project_path = r"D:\Downloads\projectù"
yolo_path = os.path.join(project_path, "licence plates.v1i.yolov11")
datasetninja_path = os.path.join(project_path, "tunisian-licensed-plates-DatasetNinja")

# Nouveau dossier général
general_path = os.path.join(project_path, "merged_dataset")
os.makedirs(general_path, exist_ok=True)

# Sous-dossiers images et labels
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(general_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(general_path, split, "labels"), exist_ok=True)

# -----------------------------
# Fonction pour convertir JSON DatasetNinja -> YOLO
# -----------------------------
def json_to_yolo(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    yolo_lines = []

    img_w = data['size']['width']
    img_h = data['size']['height']

    for obj in data['objects']:
        class_id = 0  # license plate
        x1, y1 = obj['points']['exterior'][0]
        x2, y2 = obj['points']['exterior'][1]

        # Conversion en YOLO
        x_center = ((x1 + x2) / 2) / img_w
        y_center = ((y1 + y2) / 2) / img_h
        width = abs(x2 - x1) / img_w
        height = abs(y2 - y1) / img_h

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return "\n".join(yolo_lines)

# -----------------------------
# 1️⃣ Fusion DatasetNinja -> YOLO
# -----------------------------
for split in ["train", "test"]:
    ann_dir = os.path.join(datasetninja_path, split, "ann")
    img_dir = os.path.join(datasetninja_path, split, "img")

    if not os.path.exists(ann_dir):
        print(f"[INFO] Le dossier {ann_dir} n'existe pas, skip.")
        continue

    for file in os.listdir(ann_dir):
        if file.endswith(".json"):
            json_file = os.path.join(ann_dir, file)
            yolo_label = json_to_yolo(json_file)

            # Nom complet avant .json
            base_name = file[:-5]  # supprime juste ".json"

            # Cherche image correspondante
            img_name_jpg = base_name  # le nom contient déjà .jpg
            img_name_png = base_name.replace(".jpg", ".png")  # si c'est png

            src_img = None
            if os.path.exists(os.path.join(img_dir, img_name_jpg)):
                src_img = os.path.join(img_dir, img_name_jpg)
                dst_img_name = img_name_jpg
            elif os.path.exists(os.path.join(img_dir, img_name_png)):
                src_img = os.path.join(img_dir, img_name_png)
                dst_img_name = img_name_png
            else:
                print(f"[WARN] Image correspondante pour {file} non trouvée, skip.")
                continue

            # Copier image
            dst_img = os.path.join(general_path, split, "images", dst_img_name)
            shutil.copy2(src_img, dst_img)

            # Sauvegarder annotation YOLO
            label_file = os.path.join(general_path, split, "labels", base_name + ".txt")
            with open(label_file, "w") as f:
                f.write(yolo_label)

# -----------------------------
# 2️⃣ Copier les labels YOLO existants
# -----------------------------
# YOLO/train -> merged_dataset/train
src_img_dir = os.path.join(yolo_path, "train", "images")
src_label_dir = os.path.join(yolo_path, "train", "labels")
dst_img_dir = os.path.join(general_path, "train", "images")
dst_label_dir = os.path.join(general_path, "train", "labels")

if os.path.exists(src_img_dir):
    for file in os.listdir(src_img_dir):
        shutil.copy2(os.path.join(src_img_dir, file), os.path.join(dst_img_dir, file))
if os.path.exists(src_label_dir):
    for file in os.listdir(src_label_dir):
        shutil.copy2(os.path.join(src_label_dir, file), os.path.join(dst_label_dir, file))

# YOLO/valid -> merged_dataset/valid
src_img_dir = os.path.join(yolo_path, "valid", "images")
src_label_dir = os.path.join(yolo_path, "valid", "labels")
dst_img_dir = os.path.join(general_path, "valid", "images")
dst_label_dir = os.path.join(general_path, "valid", "labels")

if os.path.exists(src_img_dir):
    for file in os.listdir(src_img_dir):
        shutil.copy2(os.path.join(src_img_dir, file), os.path.join(dst_img_dir, file))
if os.path.exists(src_label_dir):
    for file in os.listdir(src_label_dir):
        shutil.copy2(os.path.join(src_label_dir, file), os.path.join(dst_label_dir, file))

print(f"[DONE] Fusion terminée ! Le dossier général est : {general_path}")


[DONE] Fusion terminée ! Le dossier général est : D:\Downloads\projectù\merged_dataset


In [11]:
import os

dossier = r"D:\Downloads\projectù\merged_dataset"

nb_images = len([
    f for f in os.listdir(dossier)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

print("Nombre total d’images :", nb_images)


Nombre total d’images : 0


In [26]:
import os
import json
import shutil

# -----------------------------
# Chemins du projet
# -----------------------------
project_path = r"D:\Downloads\projectù"
yolo_path = os.path.join(project_path, "licence plates.v1i.yolov11")
datasetninja_path = os.path.join(project_path, "tunisian-licensed-plates-DatasetNinja")

# Nouveau dossier général
general_path = os.path.join(project_path, "merged_dataset")
os.makedirs(general_path, exist_ok=True)

# Sous-dossiers images et labels
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(general_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(general_path, split, "labels"), exist_ok=True)

# -----------------------------
# Fonction pour convertir JSON DatasetNinja -> YOLO
# -----------------------------
def json_to_yolo(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    yolo_lines = []

    img_w = data['size']['width']
    img_h = data['size']['height']

    for obj in data['objects']:
        class_id = 0  # license plate
        x1, y1 = obj['points']['exterior'][0]
        x2, y2 = obj['points']['exterior'][1]

        # Conversion en YOLO format
        x_center = ((x1 + x2) / 2) / img_w
        y_center = ((y1 + y2) / 2) / img_h
        width = abs(x2 - x1) / img_w
        height = abs(y2 - y1) / img_h

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return "\n".join(yolo_lines)

# -----------------------------
# 1️⃣ Fusion DatasetNinja -> YOLO
# -----------------------------
for split in ["train", "test"]:
    ann_dir = os.path.join(datasetninja_path, split, "ann")
    img_dir = os.path.join(datasetninja_path, split, "img")

    if not os.path.exists(ann_dir):
        print(f"[INFO] Le dossier {ann_dir} n'existe pas, skip.")
        continue

    for file in os.listdir(ann_dir):
        if file.endswith(".json"):
            json_file = os.path.join(ann_dir, file)
            yolo_label = json_to_yolo(json_file)

            # Nom complet avant .json
            base_name = file[:-5]  # supprime ".json"

            # Cherche image correspondante
            img_name_jpg = base_name  # ex: 142.jpg
            img_name_png = base_name.replace(".jpg", ".png")

            src_img = None
            final_img_name = None
            if os.path.exists(os.path.join(img_dir, img_name_jpg)):
                src_img = os.path.join(img_dir, img_name_jpg)
                final_img_name = img_name_jpg
            elif os.path.exists(os.path.join(img_dir, img_name_png)):
                src_img = os.path.join(img_dir, img_name_png)
                final_img_name = img_name_png
            else:
                print(f"[WARN] Image correspondante pour {file} non trouvée, skip.")
                continue

            # Copier image
            dst_img = os.path.join(general_path, split, "images", final_img_name)
            shutil.copy2(src_img, dst_img)

            # Sauvegarder label YOLO avec **le même nom de base que l'image**
            label_file = os.path.join(general_path, split, "labels", os.path.splitext(final_img_name)[0] + ".txt")
            with open(label_file, "w") as f:
                f.write(yolo_label)

# -----------------------------
# 2️⃣ Copier les labels YOLO existants
# -----------------------------
def copy_yolo_split(split):
    src_img_dir = os.path.join(yolo_path, split, "images")
    src_label_dir = os.path.join(yolo_path, split, "labels")
    dst_img_dir = os.path.join(general_path, split, "images")
    dst_label_dir = os.path.join(general_path, split, "labels")

    if os.path.exists(src_img_dir):
        for file in os.listdir(src_img_dir):
            shutil.copy2(os.path.join(src_img_dir, file), os.path.join(dst_img_dir, file))
    if os.path.exists(src_label_dir):
        for file in os.listdir(src_label_dir):
            shutil.copy2(os.path.join(src_label_dir, file), os.path.join(dst_label_dir, file))

copy_yolo_split("train")
copy_yolo_split("valid")

print(f"[DONE] Fusion terminée ! Le dossier général est : {general_path}")


[DONE] Fusion terminée ! Le dossier général est : D:\Downloads\projectù\merged_dataset


In [27]:
import os

# Chemin du dossier fusionné
general_path = r"D:\Downloads\projectù\merged_dataset"

# Vérification des splits
for split in ["train", "valid", "test"]:
    images_dir = os.path.join(general_path, split, "images")
    labels_dir = os.path.join(general_path, split, "labels")

    # Compter images (.jpg ou .png)
    num_images = len([f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))])

    # Compter labels (.txt)
    num_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith(".txt")])

    print(f"[{split.upper()}] Images: {num_images}, Labels: {num_labels}")


[TRAIN] Images: 697, Labels: 1264
[VALID] Images: 41, Labels: 41
[TEST] Images: 142, Labels: 284


In [28]:
print(142*2)

284


In [29]:
import os

# Dossier test
test_labels_dir = r"D:\Downloads\projectù\merged_dataset\test\labels"
test_images_dir = r"D:\Downloads\projectù\merged_dataset\test\images"

# Lister toutes les images
image_files = [f for f in os.listdir(test_images_dir) if f.lower().endswith((".jpg", ".png"))]

# Pour chaque label dans labels/, supprimer celui qui **ne correspond pas exactement à l'image**
for label_file in os.listdir(test_labels_dir):
    if not label_file.endswith(".txt"):
        continue

    # Base du nom du label
    base_label = label_file[:-4]  # retire .txt

    # Si le label ne correspond à aucune image, on le supprime
    if base_label not in image_files:
        label_path = os.path.join(test_labels_dir, label_file)
        os.remove(label_path)
        print(f"[REMOVED] {label_file}")

print("[DONE] Labels doublons supprimés dans test.")


[REMOVED] 0.txt
[REMOVED] 1.txt
[REMOVED] 10.txt
[REMOVED] 100.txt
[REMOVED] 101.txt
[REMOVED] 102.txt
[REMOVED] 103.txt
[REMOVED] 104.txt
[REMOVED] 105.txt
[REMOVED] 106.txt
[REMOVED] 107.txt
[REMOVED] 108.txt
[REMOVED] 109.txt
[REMOVED] 11.txt
[REMOVED] 110.txt
[REMOVED] 111.txt
[REMOVED] 112.txt
[REMOVED] 113.txt
[REMOVED] 114.txt
[REMOVED] 115.txt
[REMOVED] 116.txt
[REMOVED] 117.txt
[REMOVED] 118.txt
[REMOVED] 119.txt
[REMOVED] 12.txt
[REMOVED] 120.txt
[REMOVED] 121.txt
[REMOVED] 122.txt
[REMOVED] 123.txt
[REMOVED] 124.txt
[REMOVED] 125.txt
[REMOVED] 126.txt
[REMOVED] 127.txt
[REMOVED] 128.txt
[REMOVED] 129.txt
[REMOVED] 13.txt
[REMOVED] 130.txt
[REMOVED] 131.txt
[REMOVED] 132.txt
[REMOVED] 133.txt
[REMOVED] 134.txt
[REMOVED] 135.txt
[REMOVED] 136.txt
[REMOVED] 137.txt
[REMOVED] 138.txt
[REMOVED] 139.txt
[REMOVED] 14.txt
[REMOVED] 140.txt
[REMOVED] 141.txt
[REMOVED] 15.txt
[REMOVED] 16.txt
[REMOVED] 17.txt
[REMOVED] 18.txt
[REMOVED] 19.txt
[REMOVED] 2.txt
[REMOVED] 20.txt
[REMOVED]

In [30]:
import os

# Chemin du dossier fusionné
general_path = r"D:\Downloads\projectù\merged_dataset"

# Vérification des splits
for split in ["train", "valid", "test"]:
    images_dir = os.path.join(general_path, split, "images")
    labels_dir = os.path.join(general_path, split, "labels")

    # Compter images (.jpg ou .png)
    num_images = len([f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))])

    # Compter labels (.txt)
    num_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith(".txt")])

    print(f"[{split.upper()}] Images: {num_images}, Labels: {num_labels}")


[TRAIN] Images: 697, Labels: 1264
[VALID] Images: 41, Labels: 41
[TEST] Images: 142, Labels: 142


In [35]:
import os

split = "train"
labels_dir = rf"D:\Downloads\projectù\merged_dataset\{split}\labels"
images_dir = rf"D:\Downloads\projectù\merged_dataset\{split}\images"

# Lister toutes les images
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))]

# Créer un dictionnaire pour savoir quels labels garder
labels_to_keep = {}

for img in image_files:
    # Chercher tous les fichiers de label qui contiennent le nom de l'image
    base = os.path.splitext(img)[0]
    matching_labels = [f for f in os.listdir(labels_dir) if f.startswith(base)]
    
    if not matching_labels:
        continue

    # Priorité : YOLOv8 *_rf_*.txt
    yolo8_labels = [f for f in matching_labels if "_rf" in f]
    if yolo8_labels:
        # garder le premier YOLOv8 trouvé
        labels_to_keep[img] = yolo8_labels[0]
    else:
        # sinon, garder JSON (*.jpg.txt ou *.txt)
        labels_to_keep[img] = matching_labels[0]

# Supprimer tous les labels qui ne sont pas dans labels_to_keep.values()
for label_file in os.listdir(labels_dir):
    if label_file not in labels_to_keep.values():
        os.remove(os.path.join(labels_dir, label_file))
        print(f"[REMOVED] {label_file}")

# Vérification finale
num_images = len([f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))])
num_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith(".txt")])
print(f"[{split.upper()}] Images: {num_images}, Labels: {num_labels}")


[REMOVED] 142.txt
[REMOVED] 143.txt
[REMOVED] 144.txt
[REMOVED] 145.txt
[REMOVED] 146.txt
[REMOVED] 147.txt
[REMOVED] 148.txt
[REMOVED] 149.txt
[REMOVED] 150.txt
[REMOVED] 151.txt
[REMOVED] 152.txt
[REMOVED] 153.txt
[REMOVED] 154.txt
[REMOVED] 155.txt
[REMOVED] 156.txt
[REMOVED] 157.txt
[REMOVED] 158.txt
[REMOVED] 159.txt
[REMOVED] 160.txt
[REMOVED] 161.txt
[REMOVED] 162.txt
[REMOVED] 163.txt
[REMOVED] 164.txt
[REMOVED] 165.txt
[REMOVED] 166.txt
[REMOVED] 167.txt
[REMOVED] 168.txt
[REMOVED] 169.txt
[REMOVED] 170.txt
[REMOVED] 171.txt
[REMOVED] 172.txt
[REMOVED] 173.txt
[REMOVED] 174.txt
[REMOVED] 175.txt
[REMOVED] 176.txt
[REMOVED] 177.txt
[REMOVED] 178.txt
[REMOVED] 179.txt
[REMOVED] 180.txt
[REMOVED] 181.txt
[REMOVED] 182.txt
[REMOVED] 183.txt
[REMOVED] 184.txt
[REMOVED] 185.txt
[REMOVED] 186.txt
[REMOVED] 187.txt
[REMOVED] 188.txt
[REMOVED] 189.txt
[REMOVED] 190.txt
[REMOVED] 191.txt
[REMOVED] 192.txt
[REMOVED] 193.txt
[REMOVED] 194.txt
[REMOVED] 195.txt
[REMOVED] 196.txt
[REMOVED] 

In [36]:
import os

# Chemin du dossier fusionné
general_path = r"D:\Downloads\projectù\merged_dataset"

# Vérification des splits
for split in ["train", "valid", "test"]:
    images_dir = os.path.join(general_path, split, "images")
    labels_dir = os.path.join(general_path, split, "labels")

    # Compter images (.jpg ou .png)
    num_images = len([f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))])

    # Compter labels (.txt)
    num_labels = len([f for f in os.listdir(labels_dir) if f.lower().endswith(".txt")])

    print(f"[{split.upper()}] Images: {num_images}, Labels: {num_labels}")


[TRAIN] Images: 697, Labels: 697
[VALID] Images: 41, Labels: 41
[TEST] Images: 142, Labels: 142


880